# Map matching
We have cleaned our data, now we can start our study on the taxi GPS trajectories. Our first step is to perform **map-matching** on the GPS trajectory data to get the corrected GPS coordinates and the road ID sequence for each trajectory.

### What is map matching
Map matching is the process of aligning raw GPS data points with a known road network to determine the most likely path traveled. Since GPS readings can be imprecise due to various factors, map-matching algorithms help improve accuracy by snapping these points to the road network. Techniques vary from geometric approaches to more advanced probabilistic models like Hidden Markov Models (HMM), which account for both the GPS noise and the logical connectivity of roads.

### Map matching with Valhalla's Meili
The map matching tool we'll be using is **Valhalla's Meili**. [Valhalla](https://valhalla.github.io/valhalla/) is an open source routing engine and accompanying libraries for use with OpenStreetMap data. [Meili](https://valhalla.github.io/valhalla/api/map-matching/api-reference/) is Valhalla's map matching service that aligns GPS traces to known roads or paths by comparing raw location data to OpenStreetMap's detailed road network. It can handle large-scale spatial data and is optimized for high-performance processing, making it ideal for applications like traffic analysis or location-based services.

### Setting up Valhalla on my machine
I mainly followed these two articles ([link1](https://ikespand.github.io/posts/meili/?source=post_page-----4ea6b1c88a4c--------------------------------), [link2](https://nickdoulos.com/posts/map-matching/)) to set up and run Valhalla server with docker on my Mac. 

# Open Street Map

# Map matching demonstration
In this project, we'll use map matching for two purposes:
- get GPS trace that are snapped to the actual roads
- get the trajectory sequence (the sequence of way IDs) of each trip

Let's go through the process of map matching for the first taxi GPS trace with Valhalla's Meili for demonstration. 

In [28]:
import pandas as pd

# Read clean data file
df = pd.read_pickle('clean_data.pkl')
df.head()

,TRIP_ID,POLYLINE
10,1372636875620000233,"[[-8.619894,41.148009],[-8.620164,41.14773],[-..."
13,1372638595620000233,"[[-8.608716,41.153499],[-8.607627,41.153481],[..."
25,1372639092620000233,"[[-8.632737,41.168295],[-8.6328,41.16825],[-8...."
63,1372641600620000612,"[[-8.585811,41.148666],[-8.585757,41.148972],[..."
64,1372636896620000360,"[[-8.617599,41.146137],[-8.617581,41.14593],[-..."


In [29]:
# Get the first taxi GPS trace, and convert it into a dataframe
trace = df.iloc[0]['POLYLINE']
trace = trace[2:-2].split('],[')
trace = pd.DataFrame([(float(p.split(',')[1]), float(p.split(',')[0])) for p in trace], columns=['lat', 'lon'])

In [30]:
trace.head()

,lat,lon
0,41.148009,-8.619894
1,41.147730,-8.620164
2,41.148513,-8.620650
3,41.150313,-8.620920
4,41.151951,-8.621208


### Send a request to Meili
From Valhalla's doc,   
"There are two separate Map Matching calls that perform different operations on an input set of latitude, longitude coordinates. The `trace_route` action returns the shape snapped to the road network and narrative directions, while `trace_attributes` returns detailed attribution along the portion of the route.  
It is important to note that all service requests should be **POST** because `shape` or `encoded_polyline` can be fairly large."

To get the sequence of way IDs, we'll use the `trace_attributes` action, and we'll prepare out POST request according to the format it requires.

In [31]:
# Prepare the JSON payload for POST requests for the trace_routes action
# Convert the prepared dataframe into a JSON
meili_coordinates = trace.to_json(orient='records')

# Providing needed data for the body of Meili's request
meili_head = '{"shape":'
# Parameters of the request
meili_tail = ""","search_radius": 150, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}"""

# Combining all the string into a single request
meili_request_body = meili_head + meili_coordinates + meili_tail

In [32]:
# Send the request
import requests

# The URL of our local Valhalla server
# We're using the 'trace_attributes' action
url = "http://localhost:8002/trace_attributes"

# Providing headers to the request
headers = {'Content-type': 'application/json'}

# Sending a request
r = requests.post(url, data=str(meili_request_body), headers=headers)

In [33]:
import json

# Checking if the response from Meili was correct
if r.status_code == 200:
    # Parsing the JSON response
    response_text = json.loads(r.text)
else:
    print("Incorrect request, status code {}".format(r.status_code))

`response_text` contains all information returned by Meili, and the information we're interested in are: the coordinates of the points snapped to the road, which is stored in the item `matched_points`, and the sequence of way IDs of the trips, which is stored in the list of edges.

In [34]:
# Get the list of matched points and convert it into a dataframe
matched_trace = response_text['matched_points']
matched_trace = pd.DataFrame([(p['lat'], p['lon']) for p in matched_trace], columns=['lat_matched', 'lon_matched'])

# Combine it with the dataframe of the original GPS trace
trace = pd.concat([trace, matched_trace], axis=1)
trace.head()

,lat,lon,lat_matched,lon_matched
0,41.148009,-8.619894,41.148011,-8.619900
1,41.147730,-8.620164,41.147719,-8.620153
2,41.148513,-8.620650,41.148513,-8.620646
3,41.150313,-8.620920,41.150301,-8.621013
4,41.151951,-8.621208,41.151921,-8.621124


In [ ]:
# Get matched path by decoding the `shape` item
# Get sequence of way ID

### Visualizion
Let's visualize the map-matching result. The original GPS trace is plot in red line and markers, the matched GPS trace in green.

In [52]:
import folium

# Initalize the map
m = folium.Map(location=(trace.iloc[0]['lat'],trace.iloc[0]['lon']), zoom_start=15)

In [55]:
# Plot the original GPS trace on the map
folium.PolyLine(trace[['lat','lon']], color='red', opacity=0.5, weight=4).add_to(m)

# Add circle markers for each point on the GPS trace
for index, p in trace.iterrows():
    folium.CircleMarker(location=(p['lat'], p['lon']), 
                        radius=4, weight=0, fill_color='red', fill_opacity=1).add_to(m)

# Plot the matched GPS trace on the map, and add circle markers for each point
folium.PolyLine(trace[['lat_matched','lon_matched']], color='green', opacity=0.5,weight=4).add_to(m)

for index, p in trace.iterrows():
    folium.CircleMarker(location=(p['lat_matched'], p['lon_matched']), 
                        radius=4, weight=0, fill_color='green', fill_opacity=1).add_to(m)

# Plot the matched path on the map

# Add layer control to the map

In [56]:
m